In [116]:
input_file = "16_input.txt" 

In [117]:
import math

In [118]:
valves = []

with open(input_file) as f:
    for line in f:
        words = line.rstrip().split()
        rate = int(words[4].rstrip(';').split('=')[-1])
        neighbours = [l.rstrip(',') for l in words[9:]]
        valves.append([words[1], rate, neighbours])

In [127]:
class Valve:
    def __init__(self, name: str, rate: int, neighbours: list[str]) -> None:
        self.name = name
        self.rate = rate
        self.neighbour_names = neighbours
        self.open = False


class PressureNetwork:
    def __init__(self, valve_list, start="AA"):
        self.valves = dict()
        for name, rate, neigh in valve_list:
            self.valves[name] = Valve(name, rate, neigh)
        for valve in self.valves.values():
            valve.neighbours = [self.valves[l] for l in valve.neighbour_names]
        self.start_name = start
        self.start_valve = self.valves[self.start_name]
        self.positive_valves = {k: v for k, v in self.valves.items() if v.rate > 0}
        self.compute_distances()
        self.rate_dict = {k: v.rate for k, v in self.positive_valves.items()}
    
    
    def compute_distances(self):
        self.dist = {v: {u: math.inf for u in self.valves} for v in self.valves}
        for v in self.valves:
            self.dist[v][v] = 0
            for u in self.valves[v].neighbour_names:
                self.dist[v][u] = 1
        
        for w in self.valves:
            for v in self.valves:
                for u in self.valves:
                    self.dist[v][u] = min(self.dist[v][u], self.dist[v][w] + self.dist[w][u])
        
        
        self.dist_rv = {v: self.dist[self.start_name][v] for v in self.positive_valves}
        self.dist_small = {v: {u: self.dist[v][u] for u in self.positive_valves} for v in self.positive_valves}
    
    
    def greedy(self):
        pressure = 0
        closed_valves = list(self.positive_valves.keys())
        current = 'AA'
        k = 0
        while k < 30 and len(closed_valves) > 0:
            maxp = -math.inf
            for v in closed_valves:
                distance = self.dist[current][v]
                rate = self.rate_dict[v]
                newp = (30 - k - distance - 1) * rate
                print(v, rate, newp)
                if newp > maxp:
                    maxp = newp
                    maxv = v
            print(maxv)
            pressure += maxp
            k -= (self.dist[current][maxv] + 1)
            closed_valves.remove(maxv)
            current = maxv
        return pressure
    
    @property
    def closed_valves(self):
        return {k: v for k, v in self.positive_valves.items() if not(v.open)}
    

In [125]:
volcano = PressureNetwork(valves)

In [126]:
print(volcano.dist_rv)
volcano.dist_small

{'JY': 10, 'FX': 4, 'VF': 6, 'KR': 2, 'MC': 3, 'AF': 3, 'XZ': 8, 'EX': 4, 'PK': 6, 'EM': 3, 'RW': 6, 'XL': 5, 'ED': 6, 'JS': 2, 'DI': 8}


{'JY': {'JY': 0,
  'FX': 13,
  'VF': 10,
  'KR': 12,
  'MC': 7,
  'AF': 13,
  'XZ': 2,
  'EX': 14,
  'PK': 15,
  'EM': 12,
  'RW': 13,
  'XL': 9,
  'ED': 4,
  'JS': 12,
  'DI': 12},
 'FX': {'JY': 13,
  'FX': 0,
  'VF': 3,
  'KR': 2,
  'MC': 6,
  'AF': 4,
  'XZ': 11,
  'EX': 5,
  'PK': 2,
  'EM': 4,
  'RW': 2,
  'XL': 7,
  'ED': 9,
  'JS': 4,
  'DI': 5},
 'VF': {'JY': 10,
  'FX': 3,
  'VF': 0,
  'KR': 5,
  'MC': 3,
  'AF': 7,
  'XZ': 8,
  'EX': 8,
  'PK': 5,
  'EM': 7,
  'RW': 3,
  'XL': 5,
  'ED': 6,
  'JS': 7,
  'DI': 2},
 'KR': {'JY': 12,
  'FX': 2,
  'VF': 5,
  'KR': 0,
  'MC': 5,
  'AF': 2,
  'XZ': 10,
  'EX': 4,
  'PK': 4,
  'EM': 2,
  'RW': 4,
  'XL': 5,
  'ED': 8,
  'JS': 2,
  'DI': 7},
 'MC': {'JY': 7,
  'FX': 6,
  'VF': 3,
  'KR': 5,
  'MC': 0,
  'AF': 6,
  'XZ': 5,
  'EX': 7,
  'PK': 8,
  'EM': 5,
  'RW': 6,
  'XL': 2,
  'ED': 3,
  'JS': 5,
  'DI': 5},
 'AF': {'JY': 13,
  'FX': 4,
  'VF': 7,
  'KR': 2,
  'MC': 6,
  'AF': 0,
  'XZ': 11,
  'EX': 3,
  'PK': 6,
  'EM': 2,
  'RW':

In [108]:
volcano.greedy()

BB 13 364
CC 2 54
DD 20 560
EE 3 81
HH 22 528
JJ 21 567
JJ
BB 13 377
CC 2 56
DD 20 580
EE 3 84
HH 22 550
DD
BB 13 442
CC 2 70
EE 3 105
HH 22 704
HH
BB 13 455
CC 2 72
EE 3 114
BB
CC 2 94
EE 3 135
EE
CC 2 100
CC


2541

In [96]:
volcano.rate_dict

{'BB': 13, 'CC': 2, 'DD': 20, 'EE': 3, 'HH': 22, 'JJ': 21}